### 🛠️ Setup

Import packages

In [3]:
import os, subprocess, sys

WD_DATE = "20230619"
# WD_DATE = "20211213"
WP_DATE = "20230620"
DBP_DATE = "20221201"

os.environ["WD_DIR"] = os.path.expanduser(f"~/kgdata/wikidata/{WD_DATE}")
os.environ["WP_DIR"] = os.path.expanduser(f"~/kgdata/wikipedia/{WP_DATE}")
os.environ["DBP_DIR"] = os.path.expanduser(f"~/kgdata/dbpedia/{DBP_DATE}")

from kgdata.config import init_dbdir_from_env
from kgdata.dataset import import_dataset

init_dbdir_from_env()

2023-08-25 11:23:59.642 | INFO     | kgdata.config:init_dbdir_from_env:23 - Wikidata directory: /nas/home/binhvu/kgdata/wikidata/20230619
2023-08-25 11:23:59.644 | INFO     | kgdata.config:init_dbdir_from_env:29 - DBpedia directory: /nas/home/binhvu/kgdata/dbpedia/20221201
2023-08-25 11:23:59.646 | INFO     | kgdata.config:init_dbdir_from_env:37 - Wikipedia directory: /nas/home/binhvu/kgdata/wikipedia/20230620


In [3]:
from sm.prelude import M

### Main

#### Test multi-lingual string compression

In [3]:
ds = import_dataset('wikidata.entity_metadata')

In [4]:
ds.get_rdd().count()

2023-08-25 08:46:07.421 | DEBUG    | kgdata.spark:get_spark_context:90 - Spark Options: [('spark.master', 'spark://ckg04.isi.edu:17077'), ('spark.executor.memory', '600g'), ('spark.driver.memory', '32g'), ('spark.driver.maxResultSize', '128g')]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/25 08:46:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


103326088

In [7]:
def compress_multilingual_string(s):
    values = []
    existing_values = {}
    lang2index = {}
    for lang, value in s.lang2value.items():
        if value not in existing_values:
            values.append(value)
            existing_values[value] = len(values) - 1
        lang2index[lang] = existing_values[value]
    
    return [values, lang2index]
    return {
        "values": values,
        "lang2index": lang2index,
        "lang": s.lang
    }

In [30]:
import orjson

def measure(label):
    size = len(orjson.dumps(label.to_dict()))
    size2 = len(orjson.dumps(compress_multilingual_string(label)))
    
    return size, size2
    
res = ds.get_rdd().map(lambda e: e.label).map(measure).fold((0, 0), lambda t1, t2: (t1[0] + t2[0], t1[1] + t2[1]))
print(M.percentage(res[1], res[0], M.datasize))

62.92% (17.0GiB/27.0GiB)


----------------------------------------                                        
Exception occurred during processing of request from ('127.0.0.1', 48154)
Traceback (most recent call last):
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/data/binhvu/anaconda3/envs/py311/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/data/binhvu/gramsplus/.venv/lib/python3.11/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/data/binhvu/gramsplus/.venv/lib/python3.11/site-packages/pyspark/accumul

In [29]:
res = ds.get_rdd().map(lambda e: e.description).map(measure).fold((0, 0), lambda t1, t2: (t1[0] + t2[0], t1[1] + t2[1]))
print(M.percentage(res[1], res[0], M.datasize))

95.05% (95.7GiB/100.7GiB)
